# Investigating Impact of *Crash Location's Relation to Roadway* on Probability and Odds of Roundabout vs. Intersection Crash

In [18]:
# install dependencies
!pip install numpy matplotlib scipy pandas seaborn pingouin

You should consider upgrading via the 'D:\conda\python.exe -m pip install --upgrade pip' command.


In [1]:
# import directives
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import ttest_rel
import pingouin as pt

In [9]:
# Preprocessing -- should be general-purpose
# load data
intersection = pd.read_csv("relation_intersection.csv").replace(',','', regex=True).fillna(0)
roundabout = pd.read_csv("relation_roundabout.csv").replace(',','', regex=True).fillna(0)

# Parse crash year and date
month_map = {
    "January": "1",
    "February": "2",
    "March": "3",
    "April": "4",
    "May": "5",
    "June": "6",
    "July": "7",
    "August": "8",
    "September": "9",
    "October": "10",
    "November": "11",
    "December": "12"
}
intersection['Crash Month'] = intersection['Crash Month'].map(month_map)
intersection['Crash Date'] = pd.to_datetime(intersection['Crash Year'] + "/" + intersection['Crash Month'] + "/01")
intersection.drop(['Crash Year', 'Crash Month'], axis=1, inplace=True)
roundabout['Crash Month'] = roundabout['Crash Month'].map(month_map)
roundabout['Crash Date'] = pd.to_datetime(roundabout['Crash Year'] + "/" + roundabout['Crash Month'] + "/01")
roundabout.drop(['Crash Year', 'Crash Month'], axis=1, inplace=True)

In [10]:
# Drop unnecessary columns
intersection = intersection.filter(['Crash Date', 'In the Median', 'Total'])
roundabout = roundabout.filter(['Crash Date', 'In the Median', 'Total'])

# Type conversion
intersection['In the Median'] = intersection['In the Median'].astype(float)
intersection['Total'] = intersection['Total'].astype(float)
roundabout['In the Median'] = roundabout['In the Median'].astype(float)
roundabout['Total'] = roundabout['Total'].astype(float)

In [11]:
intersection['Intersection %'] = intersection['In the Median'] / intersection['Total']
roundabout['Roundabout %'] = roundabout['In the Median'] / roundabout['Total']

intersection.drop(['In the Median', 'Total'], axis=1, inplace=True)
roundabout.drop(['In the Median', 'Total'], axis=1, inplace=True)

In [12]:
# Aggregate data
aggregate = pd.merge(intersection, roundabout, on=['Crash Date'])

# save aggregated values for later and remove them from table
avg = aggregate.iloc[len(aggregate) - 1].to_dict()

In [13]:
# plot data
aggregate.plot(x='Crash Date', kind='line', ylabel="% Crashes in the Median", figsize=(8,6))
plt.savefig('median_plot.png')

In [14]:
# perform t-test on the conditional probability of the evidence's occurrence
pt.ttest(aggregate['Intersection %'], aggregate['Roundabout %'], paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.98323,60,two-sided,0.051924,"[-0.03, 0.0]",0.359958,0.872,0.789919


In [16]:
# compute Bayes' factor for impact of evidence on roundabout crash vs intersection crash odds
# print(avg['Roundabout %'] / avg['Intersection %'])

## Conclusion

Given that the $t$-test yielded the $p$-value of $0.052 > 0.05$, we fail to reject the null hypothesis, and come to the conclusion that
**the crash location's relation to the roadway** does constitute a statistically significant characteristic to the occurrence of traffic crashes in roundabouts.